# Day5 - Stacking (failed)

I wanted to try stacking with differnt combination of classifiers I tuned on day4, but my RAM is so small that kernel died frequently and I could not proceed. 

From tomorrow, I will try to investigate features, and try to remove outliers to improve the performance.

In [1]:
from sklearn import datasets
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import LinearSVC, SVC
import numpy as np
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.model_selection import GridSearchCV
import pandas as pd
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('train_final.csv')
X_pred_data = pd.read_csv('test_final.csv')
X_pred = np.asarray(X_pred_data.iloc[:, 1:25]).reshape(-1, 24)

In [3]:
X = np.asarray(data.iloc[:, 2:26]).reshape(-1, 24)
y = np.asarray(data.iloc[:, 1]).reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [4]:
xgb1 = xgb.XGBClassifier(colsample_bytree=0.65, gamma=0.1,
       learning_rate=0.05, max_depth=5,
       min_child_weight=5, n_estimators=100, objective='binary:logistic',
      reg_alpha=1, reg_lambda=1,
      subsample=0.85, verbosity=1, n_jobs=-1)

In [5]:
rf = RandomForestClassifier(n_estimators = 100, max_depth = 25, n_jobs=-1)

In [6]:
knn =  KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski', n_jobs=-1, n_neighbors=10, p=1)

In [7]:
#---------- This crasher kernel ----------
#def stacking(num_iter=10, xgb=XGBClassifier(), rf=RandomForestClassifier, knn=KNeighborsClassifier):
#    X_trains = [X_train]
#    X_tests = [X_test]
#    X_preds = [X_pred]
#    for i in range(num_iter):
#        print("---------- iteration %d ----------" %(i+1))
#        current_model = None
#        if(i % 3 == 0): 
#            current_model = xgb
#        if(i % 3 == 1):
#            current_model = rf
#        if(i % 3 == 2): 
#            current_model = knn
#        current_model.fit(X_trains[i], y_train.ravel())
#        X_trains.append(np.hstack((X_trains[i], current_model.predict(X_trains[i]).reshape(-1, 1))))
#        X_tests.append(np.hstack((X_tests[i], current_model.predict(X_tests[i]).reshape(-1, 1))))
#        X_preds.append(np.hstack((X_preds[i], current_model.predict(X_preds[i]).reshape(-1, 1))))
#        print('Accuracy: %f' % metrics.accuracy_score(y_test, current_model.predict(X_tests[i])))
#        print('AUC: %f' %metrics.roc_auc_score(y_test, current_model.predict(X_tests[i])))
#    return X_trains[num_iter], X_tests[num_iter], X_preds[num_iter]

In [8]:
#S_train, S_test, S_pred = stacking(3, xgb, rf, knn)

In [9]:
#X_stacked1 = pd.DataFrame(rf.predict_proba(X_pred)[:, 1], columns = [names[0]])
#X_stacked1['Id'] = X_pred_data['Id']
#X_stacked1 = pd.merge(X_pred_data, X_stacked1, on='Id')
#X_stacked1
#X_stacked1.to_csv("X_stacked1.csv", index=False)

In [7]:
rf.fit(X_train, y_train.ravel())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=25, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [8]:
X_pred1 = np.hstack((X_pred, rf.predict_proba(X_pred)[:,1].reshape(-1, 1)))
X_train1 = np.hstack((X_train, rf.predict_proba(X_train)[:,1].reshape(-1, 1)))
X_test1 = np.hstack((X_test, rf.predict_proba(X_test)[:,1].reshape(-1, 1)))

In [9]:
X_pred = None
X_train = None
X_test = None

In [10]:
knn.fit(X_train1, y_train.ravel())

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=10, p=1,
           weights='uniform')

In [11]:
X_pred2 = np.hstack((X_pred1, knn.predict_proba(X_pred1)[:,1].reshape(-1, 1)))
X_train2 = np.hstack((X_train1, knn.predict_proba(X_train1)[:,1].reshape(-1, 1)))
X_test2 = np.hstack((X_test1, knn.predict_proba(X_test1)[:,1].reshape(-1, 1)))

In [12]:
X_pred1 = None
X_train1 = None
X_test1 = None

In [ ]:
xgb1.fit(X_train2, y_train.ravel())

In [ ]:
X_pred2.shape

In [ ]:
rf.predict_proba(X_train)[:,1].reshape(-1, 1)